# Using VGG16 to solve Kaggle's Dogs vs Cats problem

In [7]:
%matplotlib inline

In [8]:
# path = ""
path = "sample/"

A few basic libraries that we'll need for the initial exercises:

In [9]:
from __future__ import division,print_function

import os, json, csv
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

We have created a file most imaginatively called 'utils.py' to store any little convenience functions we'll want to use. We will discuss these as we use them.

In [10]:
from imp import reload
import utils; reload(utils)
from utils import plots

# Use a pretrained VGG model with our **Vgg16** class

Our first step is simply to use a model that has been fully created for us, which can recognise a wide variety (1,000 categories) of images. We will use 'VGG', which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (*VGG  19*) and a smaller, faster model (*VGG 16*). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

We have created a python class, *Vgg16*, which makes using the VGG 16 model very straightforward. 

## The punchline: state of the art custom model in 7 lines of code

Here's everything you need to do to get >97% accuracy on the Dogs vs Cats dataset - we won't analyze how it works behind the scenes yet, since at this stage we're just going to focus on the minimum necessary to actually do useful work.

In [11]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [12]:
# Import our class, and instantiate
from vgg16 import Vgg16

## Use Vgg16 for basic image recognition

In [13]:
vgg = Vgg16()

In [14]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)
test_batches = vgg.get_batches(path+'test', batch_size=batch_size, shuffle=False)
file_ids = list(map(lambda x: x.split("/")[1].replace(".jpg", ""), test_batches.filenames))

Found 88 images belonging to 2 classes.
Found 128 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


In [15]:
vgg.finetune(batches)

In [16]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
88/88 [==============================] - 215s - loss: 2.4751 - acc: 0.5682 - val_loss: 3.5816 - val_acc: 0.7734


In [17]:
predictions = vgg.model.predict_generator(test_batches, len(test_batches.filenames), max_q_size=batch_size*2)

array([[  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   9.7317e-24],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  0.0000e+00,   1.0000e+00],
       [  9.1529e-35,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  8.7352e-33,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   5.4480e-11],
       [  1.7348e-06,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00],
       [  0.0000e+00,   1.0000e+00],
       [  1.0000e+00,   0.0000e+00],
 

In [52]:
csvfile = open('submission.csv', 'w')
writer = csv.writer(csvfile, delimiter=",")

preds = predictions[:]
preds = np.delete(preds, 1, axis=1)
np.column_stack((file_ids, preds.flatten()))

array([['1600', '1.0'],
       ['1601', '1.0'],
       ['1602', '1.0'],
       ['1603', '1.0'],
       ['1604', '1.0'],
       ['1605', '1.0'],
       ['1606', '1.0'],
       ['1607', '1.0'],
       ['1608', '1.0'],
       ['1609', '0.0'],
       ['1610', '9.1529333832956e-35'],
       ['1611', '1.0'],
       ['1612', '8.735198437980238e-33'],
       ['1613', '1.0'],
       ['1614', '1.0'],
       ['1615', '1.0'],
       ['1616', '1.0'],
       ['1617', '1.0'],
       ['1618', '1.0'],
       ['1619', '1.0'],
       ['1620', '1.7348360188407241e-06'],
       ['1621', '1.0'],
       ['1622', '1.0'],
       ['1623', '1.0'],
       ['1624', '1.0'],
       ['1625', '0.0'],
       ['1626', '1.0'],
       ['1627', '1.0'],
       ['1628', '1.0'],
       ['1629', '0.0'],
       ['1630', '3.954898147640051e-06'],
       ['1631', '1.0'],
       ['1632', '0.0'],
       ['1633', '0.0'],
       ['1634', '1.0'],
       ['1635', '1.0'],
       ['1636', '1.0'],
       ['1637', '8.940284202392333e-43'],

In [ ]:
csvfile = open('submission.csv', 'w')
writer = csv.writer(csvfile, delimiter=",")
index = 0
batch_count = 0
results = []
print("total length: %d" % (len(file_ids)))

batch,labels = next(test_batches, None)
while(batch is not None and len(batch) > 0 and index < len(file_ids)):
    preds, idxs, labels = vgg.predict(batch, False)
    for img_class in idxs:
        writer.writerow([file_ids[index], img_class])b
        index = index + 1
    batch_count = batch_count + 1
    batch,labels = next(test_batches, None)

print("total batches: %s" % (batch_count, index))
csvfile.close()